In [1]:
import os
import requests
import re
import time

### 取得所有entities

記得一定要有decode

In [2]:
all_entities_dict_open = open("./all_entities.txt","r")
all_entities_dict = eval(all_entities_dict_open.read())

all_entities_list = []

for entity in all_entities_dict.keys():
    
    entity = entity.decode('utf-8')
    
    all_entities_list.append(entity)
    
#print(all_entities_list)
print(len(all_entities_list))

12552


## Value找尋

要注意的是value會有編碼上面的問題。

In [3]:
count = 0

#挑選的property
property_list = ['dct:subject','gold:hypernym']

#不重複做
entity_value_name_list = os.listdir("./all_entities_property_value")
entity_value_name_list_len = len(entity_value_name_list)

for entity_name in all_entities_list:

    #先判別file name
    if "/" in entity_name:
        file_name = re.sub("/","!~~~!",entity_name)
        file_name = file_name + ".txt"
        
    elif ":" in entity_name:
        file_name = re.sub(":","%~~~%",entity_name)
        file_name = file_name + ".txt"
        
    else:
        file_name = entity_name + ".txt"
        
    if file_name not in entity_value_name_list:
        
        entity_value_name_list_len+=1
        print(entity_value_name_list_len, file_name)

        property_value_dict = {}

        for property_name in property_list:

            #print(property_name)
            
            #利用服務找到value
            url_values = "http://vmdbpedia.informatik.uni-leipzig.de:8080/api/1.0.0/values"
            my_paramas = {'entities':entity_name, 'property':property_name, 'format':'JSON', 'pretty':'none', 'limit':'1000', 'offset':'0', 'key':'1234', 'oldVersion':'false'}
            property_result_json = requests.get(url_values, params = my_paramas)
                
            #注意編碼
            property_result_json.encoding = "utf-8"

            property_result_dict = eval(property_result_json.text)
            
            #解析內容
            new_property_name = property_result_dict['head']['vars'][1]

            property_result_list = property_result_dict['results']['bindings']

            value_list = []
            for property_result in property_result_list:

                #判斷有沒有找到property
                if new_property_name in property_result:

                    property_value = property_result[new_property_name]['value']

                    if property_name == "dct:subject":
                        re_property_value = re.sub(r"http://dbpedia.org/resource/Category:","",property_value)
                        re_property_value = re_property_value.encode('utf-8')

                    elif property_name == "gold:hypernym":
                        re_property_value = re.sub(r"http://dbpedia.org/resource/","",property_value)
                        re_property_value = re_property_value.encode('utf-8')

                    value_list.append(re_property_value)

            property_value_dict[property_name] = value_list



        f = open("./all_entities_property_value/"+file_name,"w")
        f.write(str(property_value_dict))
        f.close()

7050 Margaret_Thatcher.txt


SyntaxError: invalid syntax (<string>, line 1)